# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [34]:
!pip install xlsxwriter

In [28]:
import pandas as pd
import xlwings as xw
reviews = pd.read_csv('reviews_sample.csv', index_col='recipe_id')
recipes = pd.read_csv('recipes_sample.csv')

recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [41]:
# Создаем случайную выборку из 5% строк каждой таблицы
recipes_sample = recipes.sample(frac=0.05, random_state=1)
reviews_sample = reviews.sample(frac=0.05, random_state=1)
writer = pd.ExcelWriter('recipes.xlsx', engine='xlsxwriter')

# Сохраняем выборку рецептов на лист "Рецепты"
recipes_sample.to_excel(writer, sheet_name='Рецепты')

# Сохраняем выборку отзывов на лист "Отзывы"
reviews_sample.to_excel(writer, sheet_name='Отзывы')

# Закрываем объект ExcelWriter, чтобы сохранить файл Excel
writer.close()


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

time_seconds = recipes_sample['minutes'] * 60
sheet.range('G1').value = 'seconds_assign'
sheet.range('G2').options(transpose=True).value = time_seconds.tolist()

wb.app.calculation = 'manual'
wb.save()
wb.close()


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Добавляем формулу Excel для столбца seconds_formula
last_row = sheet.range('A1').expand().last_cell.row
formulas_range = sheet.range(f'H2:H{last_row}')
formulas_range.formula = '=E2*60'

# Сохраняем и закрываем книгу
wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Добавляем форматирование к столбцу seconds_assign
sheet.range('G1').api.Font.Bold = True
sheet.range('G1').api.HorizontalAlignment = -4108

# Добавляем форматирование к столбцу seconds_formula
last_row = sheet.range('A1').expand().last_cell.row
format_range = sheet.range(f'H1:H{last_row}')
format_range.api.Font.Bold = True
format_range.api.HorizontalAlignment = -4108

# Сохраняем и закрываем книгу
wb.save()
wb.close()


6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Получаем диапазон значений столбца minutes
last_row = sheet.range('A1').expand().last_cell.row
minutes_range = sheet.range(f'I2:I{last_row}')

# Применяем условное форматирование к столбцу minutes
for cell in minutes_range:
    try:
        if cell.value and str(cell.value).strip():  # Проверяем, что значение не пустое
            time = int(cell.value)
            if time < 5:
                cell.color = (0, 255, 0)  # Зеленый
            elif 5 <= time <= 10:
                cell.color = (255, 255, 0)  # Желтый
            else:
                cell.color = (255, 0, 0)  # Красный
    except Exception as e:
        print(f"Ошибка в ячейке {cell.address}: {e}")

# Сохраняем и закрываем книгу
wb.save()
wb.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

# Добавляем заголовок для нового столбца
sheet.range('H1').value = 'n_reviews'

# Применяем формулу COUNTIF для подсчета количества отзывов для каждого рецепта
for i in range(len(recipes_sample)):
    recipe_id = recipes_sample.iloc[i]['id']
    count_formula = f'=COUNTIF(Отзывы!A:A, {recipe_id})'
    cell_range = f'H{i+2}'  # Начиная с 2 строки, так как у нас есть заголовок
    sheet.range(cell_range).formula = count_formula

wb.save()
wb.close()


## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 